### Import Dependencies

### Import the API key.

In [50]:
import pandas as pd
import gmaps
import requests
from config import g_key
gmaps.configure(api_key=g_key)

### Create DataFrame from saved CSV

In [51]:
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Albany,US,"2021, 11, 29 23:04:39",42.6001,-73.9662,31.17,68,58,1.99
1,1,Umm Kaddadah,SD,"2021, 11, 29 23:04:39",13.6017,26.6876,76.91,38,23,12.46
2,2,Cofradia,HN,"2021, 11, 29 23:04:40",15.4000,-88.1500,76.08,87,31,2.28
3,3,Punta Arenas,CL,"2021, 11, 29 23:01:33",-53.1500,-70.9167,51.12,43,95,19.06
4,4,Kapaa,US,"2021, 11, 29 23:04:40",22.0752,-159.3190,84.18,67,40,5.99


In [52]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [53]:
# Assign the locations to an array of latitude and longitude pairs
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], dissipating=False,
                                max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Create a Percent Humidity Heatmap

In [54]:
# Heatmap of humidity %
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(i,0) for i in humidity],
                                 dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Create a Cloudiness % Heatmap

In [55]:
# Heatmap of Cloudiness %
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds,
                                 dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Create a Wind Speed Heatmap

In [56]:
# Heatmap of Wind Speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Get Vacation Criteria

In [57]:
# Ask the customer to add a minimum and maximum temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [58]:
# Filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Umm Kaddadah,SD,"2021, 11, 29 23:04:39",13.6017,26.6876,76.91,38,23,12.46
2,2,Cofradia,HN,"2021, 11, 29 23:04:40",15.4000,-88.1500,76.08,87,31,2.28
4,4,Kapaa,US,"2021, 11, 29 23:04:40",22.0752,-159.3190,84.18,67,40,5.99
7,7,Rikitea,PF,"2021, 11, 29 23:04:41",-23.1203,-134.9692,76.46,76,2,21.07
10,10,Puerto Narino,CO,"2021, 11, 29 23:04:42",-3.7703,-70.3831,78.75,97,98,2.51
14,14,Namatanai,PG,"2021, 11, 29 23:02:31",-3.6667,152.4333,84.79,67,93,11.52
17,17,Atuona,PF,"2021, 11, 29 23:04:45",-9.8000,-139.0333,78.98,76,21,22.32
27,27,Goundi,TD,"2021, 11, 29 23:04:49",9.3627,17.3660,78.10,26,44,2.24
36,36,Vaini,TO,"2021, 11, 29 23:04:52",-21.2000,-175.2000,84.36,70,20,9.22
38,38,Yabassi,CM,"2021, 11, 29 23:04:53",4.4570,9.9682,75.88,93,87,0.87


In [59]:
# null values?
preferred_cities_df.count()

City_ID       146
City          146
Country       146
Date          146
Lat           146
Lng           146
Max Temp      146
Humidity      146
Cloudiness    146
Wind Speed    146
dtype: int64

## Get Travel Destinations

In [60]:
# Create a Dataframe to store hotel names along with city, country, max temp and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Umm Kaddadah,SD,76.91,13.6017,26.6876,
2,Cofradia,HN,76.08,15.4000,-88.1500,
4,Kapaa,US,84.18,22.0752,-159.3190,
7,Rikitea,PF,76.46,-23.1203,-134.9692,
10,Puerto Narino,CO,78.75,-3.7703,-70.3831,
14,Namatanai,PG,84.79,-3.6667,152.4333,
17,Atuona,PF,78.98,-9.8000,-139.0333,
27,Goundi,TD,78.10,9.3627,17.3660,
36,Vaini,TO,84.36,-21.2000,-175.2000,
38,Yabassi,CM,75.88,4.4570,9.9682,


In [61]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [62]:
# Iterate through the dataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    # add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    # grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [63]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Umm Kaddadah,SD,76.91,13.6017,26.6876,
2,Cofradia,HN,76.08,15.4000,-88.1500,COL.30 DE ENERO
4,Kapaa,US,84.18,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
7,Rikitea,PF,76.46,-23.1203,-134.9692,People ThankYou
10,Puerto Narino,CO,78.75,-3.7703,-70.3831,Eware Refugio Amazonico
14,Namatanai,PG,84.79,-3.6667,152.4333,Seagulls Inn Namatani
17,Atuona,PF,78.98,-9.8000,-139.0333,Villa Enata
27,Goundi,TD,78.10,9.3627,17.3660,
36,Vaini,TO,84.36,-21.2000,-175.2000,Keleti Beach Resort
38,Yabassi,CM,75.88,4.4570,9.9682,MOTEL FOMI


## Create a Maximum Temperature Heatmap from a hotel DataFrame

In [64]:
# Add template for pop up info window
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [65]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(i, 0) for i in max_temp], dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))